# Session 11 - Generative language models for zero-shot learning

In [9]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

We're going to be working with ```FLAN-T5```, a text-to-text model developed by Google. ```FLAN-T5``` is based on ```T5```, which we saw in the lecture, but it has been further finetuned on a range of common text-to-text tasks. This means that it can already perform a lot of the kinds of tasks that people use generative language models for. You can read more in the paper here: [https://arxiv.org/pdf/2210.11416.pdf](https://arxiv.org/pdf/2210.11416.pdf)

We load the model from ```huggingface```. We're here using the ```Large``` version, but you can try the other sizes if you want. The ```Large``` version is already 3.4GB, and even larger models will take a long time to download and to run - but they should also see a marked performance improvement.
 
You can read more about the available models here: [https://huggingface.co/docs/transformers/model_doc/flan-t5](https://huggingface.co/docs/transformers/model_doc/flan-t5)

In [10]:
model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


We also download and initalize the pretrained tokenizer that fits with our model:

In [3]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

## Prompt engineering

Our goal is to use the knowledge of language that FLAN-T5 has already acquired during training and to use that knowledge in different domains *without any further fine-tuning*. This is an example of what is called *zero-shot* learning.

In order for zero-shot learning to be successful, our prompts need to be carefully designed. FLAN-T5 (and similar models) are a bit less flexible than, for example, ChatGPT.

In [11]:
# classification
prompt = "classify the following text as positive or negative: I absolutely hated this movie"

# translation
#prompt = "translate from English to French: how old are you?"

# question answering
#prompt = "answer the following question: how is cheese made?"

# named entity recognition
#prompt = "find all location entities in this text: Ross comes from Scotland"

We then pass our text prompt to the tokenizer, defing some extra arguments such as the ```max_length``` of our input (anything longer than this will be truncated):

In [12]:
inputs = tokenizer(prompt, 
                    max_length = 200,
                    return_tensors="tf")

We then pass all of our input prompt tokens to the model and use than to generate an appropriate output from what FLAN-T5 has learned during training.

In [13]:
outputs = model.generate(**inputs)

In [14]:
print(tokenizer.batch_decode(outputs, 
                            skip_special_tokens=True))

['negative']


We can also do this a bit more cleverly by using a single prompt plus a F-string:

In [15]:
prompt = f"classify the following text as positive or negative: {input_text}"

NameError: name 'input_text' is not defined

This means we could, for example, write functions for specific tasks:

In [16]:
def classifier(input_text:str) -> str:
    prompt = f"classify the following text as positive or negative: {input_text}"
    inputs = tokenizer(prompt, 
                    max_length = 200,
                    return_tensors="tf")
    outputs = model.generate(**inputs)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

In [17]:
classifier("I absolutely hated this movie!")

['negative']


## Tasks

Look through previous notebooks, exercises, and datasets from Language Analytics so far this semester. In small groups, try using either ```Flan-T5``` and **ChatGPT** (or both) try to solve those problems using generative language models.

So that would mean, for example:

    - Grammatical analysis
    - Named entity recognition/extraction
    - Classification
    - Topic modelling

As an illustrative example: try using Flan-T5 to perform classification on the Fake or Real News dataset. How does it perform on ground truth? Is it better or worse than the other models we've seen? Be creative!


In [22]:
# topic moddelling

def classifier(input_text:str) -> str:
    prompt = f"assign text to one of four topics (animals, transport, food, places) {input_text}"
    inputs = tokenizer(prompt, 
                    max_length = 200,
                    return_tensors="tf")
    outputs = model.generate(**inputs)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True))


classifier("I absolutely hated this lady!")

['places']


In [25]:
# classify 

def classifier(input_text:str) -> str:
    prompt = f"classify this article as real or fake:  {input_text}"
    inputs = tokenizer(prompt, 
                    max_length = 200,
                    return_tensors="tf")
    outputs = model.generate(**inputs)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True))


classifier("Yesterday a fire broke out in copenhagen. No people were harmed")

['real']
